# Melt basics

In [5]:
import MagmaPandas as mp

Import some melt data from a csv file

In [6]:
melt_file = "./data/melts.csv"

melt = mp.read_melt(melt_file, index_col=["name"])
melt.head()

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,CO2,H2O,F,S,Cl,total
name,,,,,,,,,,,,,,,,
PI032-04-01,49.402603,14.539062,6.509044,10.058735,9.757536,3.737374,0.697769,0.132497,2.548581,0.289823,0.628502,1.440208,0.078857,0.145452,0.033958,100
PI032-04-02,48.832207,15.116896,6.508618,9.688642,9.927146,3.564377,0.940308,0.135905,2.652552,0.336327,0.672345,1.330865,0.083879,0.165394,0.044540,100
PI041-02-02,49.124916,16.950138,4.861451,9.183852,10.238358,3.788617,1.077791,0.153258,2.804343,0.561031,0.463920,0.655494,0.047654,0.068020,0.021157,100
PI041-03-01,45.941086,15.753453,6.909844,10.996510,10.778833,3.346468,1.144852,0.112117,3.153303,0.551992,0.797715,0.293341,0.079499,0.086863,0.054123,100
PI041-03-03,45.313339,15.909531,6.831676,11.175276,10.966834,3.417548,1.167550,0.088899,3.262463,0.517891,0.821116,0.311131,0.081713,0.079495,0.055540,100


On top of the regular MagmaFrame methods, Melt has additional methods for calculating

- NBO/T,
- melt densities,
- liquidus temperatures,
- Fe3+/Fe2+ ratios,
- melt FeO and Fe2O3,
- olivine-melt Fe-Mg partitioning coefficients and
- H2O-CO2 saturation pressures.

Let's check the current configuration of MagmaPandas ...

In [7]:
print(mp.configuration)


################ MagmaPandas ################
#############################################
General settings_____________________________
fO2 buffer................................QFM
Δ fO2.......................................1
Melt Fe3+/Fe2+........................Sun2024
Kd Fe-Mg ol-melt.......................toplis
Melt thermometer...............putirka2008_15
Volatile solubility model......IaconoMarziano
Volatile species........................mixed


... and calculate liquidus temperatures with a melt-only thermometer at 4 kbar. The configuration is currently set to use equation 15 from Putirka (2008).

In [12]:
P_bar = 4e3

temperatures = melt.temperature(P_bar=P_bar)
temperatures

name
PI032-04-01    1443.385914
PI032-04-02    1445.564297
PI041-02-02    1412.625935
PI041-03-01    1470.801711
PI041-03-03    1469.717871
PI041-05-04    1392.562896
PI041-05-06    1417.312468
PI041-07-01    1458.806150
PI041-07-02    1463.178696
PI052-01-02    1381.292654
Name: T_K, dtype: float64

We can switch to a different thermometer by changing the configuration:

In [ ]:
mp.configuration.melt_thermometer = "putirka2008_16"

temperatures_new = melt.temperature(P_bar=P_bar)
temperatures_new

With these temperatures we can calculate melt Fe3+/Fe2+ ratios. The configuration is currently set to fO2 buffered at QFM+1 and the Borisov et al. (2018) Fe3+/Fe2+ model.

In [ ]:
melt.head()

In [ ]:
print(mp.configuration)
mp.configuration.dfO2 = 2

In [ ]:
moles = melt.moles

In [ ]:
moles.FeO_Fe2O3_calc(0.33, wtpc=False)

In [ ]:
mp.configuration.Fe3Fe2_model = "borisov"
Fe3Fe2_QFM1 = melt.Fe3Fe2(T_K=temperatures, P_bar=P_bar)
Fe3Fe2_QFM1

Changing the fO2 buffer to lower log offset values reduces the amount of Fe3+

In [ ]:
mp.configuration.dfO2 = 0

melt.Fe3Fe2(T_K=temperatures, P_bar=P_bar)

With our Fe3+/Fe2+ ratios we can calculate melt FeO and Fe2O3 contents

In [ ]:
melt.FeO_Fe2O3_calc(Fe3Fe2=Fe3Fe2_QFM1)

Our melts also contain H2O and CO2, so let's calculate volatile saturation pressures. In the configuration we see that we're currently using the Iacono-Marziano et al. (2012) model with mixed CO2-H2O fluids:

In [ ]:
print(mp.configuration)

In [ ]:
pressures_IM2012 = melt.volatile_saturation_pressure(T_K=temperatures)
pressures_IM2012

Lets change the volatile solubility model to Allison et al. (2022):

In [ ]:
mp.configuration.volatile_solubility = 'Allison2022'
print(mp.configuration)

In [ ]:
pressures_A22 = melt.volatile_saturation_pressure(T_K=temperatures)
pressures_A22